# Knowledge Representation on the Web -- RDF tutorial

In this tutorial we'll learn the basics of interacting with RDF graphs with Python. We'll be using rdflib for this, a widely used Ptyhon library for RDF (all documentation can be found [here](https://rdflib.readthedocs.io/en/stable/index.html))

1. can not parse "https://csarven.ca/" -> add format="n3"
2. hadAge not in foaf namespace -> changed to foaf:age
3. Teacher not in foaf namespace -> changed to TEACH:Teacher, and added a section on adding a new namespace

## Imports
These are the main classes and types we'll be using from rdflib

In [2]:
import sys
!{sys.executable} -m pip install rdflib

from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF

import pprint


## Loading data remotely and from files

rdflib accepts importing RDF data from a variety of sources, either locally from a file (including an extensive support of serializations), or remotely via a URI (this is a great way of checking practically if URIs return RDF according to the 3rd Linked Data principle).

A Graph object is always required to load triples.
**Note**: to load quads, and hence supporting named graphs, you'll need to use an instance of ConjunctiveGraph instead


In [5]:
g = Graph()
h = Graph()
f = Graph()

result = g.parse("http://www.w3.org/People/Berners-Lee/card")

result2 = h.parse("https://csarven.ca/", format="n3")

result3 = f.parse("../data/ingredients.rdf")

print("Graph has %s statements." % len(g))
print("Graph has %s statements." % len(h))
print("Graph has %s statements." % len(f))


Graph has 86 statements.
Graph has 522 statements.
Graph has 837 statements.


In [18]:
v = g.serialize(format="xml")

print(v.decode("utf-8"))

<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF
   xmlns="http://xmlns.com/foaf/0.1/"
   xmlns:cc="http://creativecommons.org/ns#"
   xmlns:cert="http://www.w3.org/ns/auth/cert#"
   xmlns:con="http://www.w3.org/2000/10/swap/pim/contact#"
   xmlns:dc="http://purl.org/dc/elements/1.1/"
   xmlns:dct="http://purl.org/dc/terms/"
   xmlns:doap="http://usefulinc.com/ns/doap#"
   xmlns:geo="http://www.w3.org/2003/01/geo/wgs84_pos#"
   xmlns:ldp="http://www.w3.org/ns/ldp#"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
   xmlns:schema="http://schema.org/"
   xmlns:sioc="http://rdfs.org/sioc/ns#"
   xmlns:solid="http://www.w3.org/ns/solid/terms#"
   xmlns:space="http://www.w3.org/ns/pim/space#"
   xmlns:vcard="http://www.w3.org/2006/vcard/ns#"
>
  <rdf:Description rdf:about="https://www.w3.org/People/Berners-Lee/card#i">
    <con:office rdf:nodeID="Nc56c2a602c014c408f7b0adf1f4e2517"/>
    <nick>timbl</nick>
    <con:homePage rdf:r

In [ ]:
g = Graph()
g.parse("demo.ttl", format='turtle')

len(g) # prints 2

In [ ]:
for stmt in g:
    pprint.pprint(stmt)

In [ ]:
# You can also parse directly from a string
g = Graph()

g.parse(data = '<urn:a> <urn:p> <urn:b>.', format='n3')
for stmt in g:
    pprint.pprint(stmt)

## Saving RDF graphs

We use the function Graph.serialize(format)

In [ ]:
g = Graph()
g.parse("demo.nt", format='n3')
# for a,b,c in g:
#     print(a,b,c)

print(g.serialize(format='nt').decode()) # 'html', 'hturtle', 'mdata', 'microdata', 'n3', 'nquads', 'nt', 'rdfa', 'rdfa1.0', 'rdfa1.1', 'trix', 'turtle', 'xml'



##  Merging graphs

Merging graphs can be done via sequential parsings or by the overloaded operator +

**Note:** Set-theoretic graph semantics apply

In [10]:
graph = Graph()

# Sequential parsings merge *new* triples

graph.parse("../data/ingredients.rdf")
graph.parse("../data/recipes.rdf")

print("Graph has {} triples".format(len(graph)))

Graph has 1299 triples


In [22]:
g1 = Graph()
g1.parse("../data/ingredients.rdf")
print("g1 has {} triples".format(len(g1)))

g2 = Graph()
g2.parse("../data/ghostbusters.ttl", format='ttl')
print("g2 has {} triples".format(len(g2)))

union = g1 + g2
print("g1 + g2 has {} triples".format(len(union)))

print("len(g1) + len(g2) equals {}".format(len(g1)+len(g2)))

intersection = g1 & g2
print("g1 & g2 has {} triples".format(len(intersection)))

print("is the graph connected? {}".format(intersection.connected()))

g1 has 837 triples
g2 has 52337 triples
g1 + g2 has 53174 triples
len(g1) + len(g2) equals 53174
g1 & g2 has 0 triples
is the graph connected? False


In [23]:
g1 = Graph()
g1.parse("../data/ingredients.rdf")
print("g1 has {} triples".format(len(g1)))

g2 = Graph()
g2.parse("../data/recipes.rdf")
print("g2 has {} triples".format(len(g2)))

union = g1 + g2
print("g1 + g2 has {} triples".format(len(union)))

print("len(g1) + len(g2) equals {}".format(len(g1)+len(g2)))

intersection = g1 & g2
print("g1 & g2 has {} triples".format(len(intersection)))

print("is the intersection connected? {}".format(intersection.connected()))
print("is the union connected? {}".format(union.connected()))

g1 has 837 triples
g2 has 480 triples
g1 + g2 has 1299 triples
len(g1) + len(g2) equals 1317
g1 & g2 has 18 triples
is the intersection connected? True
is the union connected? False


In [34]:
g1 = Graph()
g1.parse("../demo.nt")
print("g1 has {} triples".format(len(g1)))

g2 = Graph()
g2.parse("../data/ontologies/recipes.rdf")
print("g2 has {} triples".format(len(g2)))

True

In [ ]:
# Now, if we merge graphs with different contents

tim_g = Graph()
tim_g.parse("http://www.w3.org/People/Berners-Lee/card")
print("Tim graph has {} triples".format(len(tim_g)))

g3 = g1 + tim_g
print("g3 has {} triples".format(len(g3)))


## Creating RDF triples

Triples are added to the graph with the function Graph.add()

The parameter is a triple given in a Python **tuple** (subject, predicate, object)

Notice the namespace convenience syntax!

In [ ]:
g = Graph()

# Create an identifier to use as the subject for Donna.
donna = BNode()
# donna = URIRef("http://example.org/donna")

# Add triples using store's add method.
g.add( (donna, RDF.type, FOAF.Person) )
g.add( (donna, FOAF.nick, Literal("donna", lang="foo")) )
g.add( (donna, FOAF.name, Literal("Donna Fales")) )
g.add( (donna, FOAF.mbox, URIRef("mailto:donna@example.org")) )

for i in range(100):
    g.add((donna, FOAF.age, Literal(i)))

print(len(g))

for s in g:
    pprint.pprint(s)
    print()


In [ ]:
for stmt in g:
    print(stmt)

In [ ]:
print(FOAF.Person)

In [ ]:
print(FOAF.imadethisup)

## Namespaces 
The namespace module defines many common namespaces such as RDF, RDFS, OWL, FOAF, SKOS, etc., but you can also easily add URIs within a different namespace:


In [ ]:
TEACH = Namespace("http://linkedscience.org/teach/ns#")
TEACH.Teacher

Check out the specification to see which other terms are used within the TEACH namespace. http://linkedscience.org/teach/ns/#sec-specification. 
You can use a NamespaceManager to bind a prefix to a namespace: 

In [ ]:
g.namespace_manager.bind('TEACH', URIRef('http://linkedscience.org/teach/ns#'))
TEACH.Teacher.n3(g.namespace_manager)

In [ ]:
KRW = Namespace("http://krw.vu.nl/data#")
KRW.Teacher
KRW.Student


## Navigating graphs

rdflib uses iterators to navigate Graphs. The methods for navigating subjects, predicates and objects are Graph.subjects, Graph.predicates, Graph.objects

In [ ]:
g = Graph()
g.parse("http://www.w3.org/People/Berners-Lee/card")

# Iterate over triples in store and print them out.
print("--- printing raw triples ---")
for s, p, o in g:
    print(p)

In [ ]:
# Printing subjects, predicates and objects out of the tuple omits Python datatypes
print("--- printing raw triples ---")
for s, p, o in g:
    print(s, p, o)

In [ ]:
print("PRINTING SUBJECTS")
for s in g.subjects():
    print(s)

In [ ]:
print("PRINTING PREDICATES")
for p in g.predicates():
    if len(p) > 13:
        print(p)


In [ ]:
print("PRINTING OBJECTS")
for o in g.objects():
    print(o)

We can also filter the subjects, predicates and objects we want to retrieve, and match their values like in a database "join" operation

In [ ]:
g = Graph()

# Create an identifier to use as the subject for Donna.
donna = URIRef('urn:donna')
ila = URIRef('urn:ila')
# Add triples using store's add method.
g.add( (donna, RDF.type, FOAF.Person) )
g.add( (ila, RDF.type, FOAF.Person) )
g.add( (ila, RDF.type, TEACH.Teacher) )
g.add( (donna, FOAF.nick, Literal("donna", lang="foo")) )
g.add( (donna, FOAF.name, Literal("Donna Fales")) )
g.add( (donna, FOAF.mbox, URIRef("mailto:donna@example.org")) )
g.add( (ila, FOAF.mbox, URIRef("mailto:ila@example.org")) )

# For each foaf:Person in the store print out its mbox property.
# print("--- printing mboxes ---")
# for person in g.subjects(RDF.type, FOAF.Person):
#     for mbox in g.objects(person, FOAF.mbox):
#         print(mbox)

# You can reuse matches of subjects to filter further e.g. objects
for entity in g.subjects(RDF.type, None):
    print(entity)
    for objects in g.objects(entity, RDF.type):
        print(objects)

### Basic triple matching (almost querying!)

We use method Graph.triples and a Python tuple that acts as a mask for specifying our criteria

In [ ]:
g = Graph()
g.parse("http://www.w3.org/People/Berners-Lee/card")

tim = URIRef("https://www.w3.org/People/Berners-Lee/card#i")

if ( tim, RDF.type, FOAF.Person ) in g:
   print("This graph knows that Tim is a person!")

if ( tim, None, None ) in g:
    print("This graph contains triples about Tim!")

In [ ]:
for s,p,o in g.triples( (None, None, None) ):
    print(s,p,o)

In [20]:
for s,p,o in g.triples( (tim, RDF.type, None) ):
    if(o == 'true'):
        print(s)
    

NameError: name 'tim' is not defined

## Namespaces and bindings

In [ ]:
mid_uri = URIRef("http://purl.org/midi-ld/midi#")
mid = Namespace(mid_uri)

print(mid['hello'])  # as item - for things that are not valid python identifiers
print(mid.hello)     # as attribute

In [ ]:
g = Graph()

# Create an identifier to use as the subject for Donna.
donna = BNode()

# Add triples using store's add method.
g.add( (donna, RDF.type, FOAF.Person) )
g.add( (donna, FOAF.nick, Literal("donna", lang="foo")) )
g.add( (donna, FOAF.name, Literal("Donna Fales")) )
g.add( (donna, FOAF.mbox, URIRef("mailto:donna@example.org")) )

print(g.serialize(format='turtle').decode())

In [ ]:
foaf_uri = URIRef("http://xmlns.com/foaf/0.1/")
foaf_namespace = Namespace(foaf_uri)

g = Graph()

# Bind a few prefix, namespace pairs for more readable output
g.bind("foaf", foaf_namespace)

# Create an identifier to use as the subject for Donna.
donna = BNode()

# Add triples using store's add method.
g.add( (donna, RDF.type, FOAF.Person) )
g.add( (donna, FOAF.nick, Literal("donna", lang="foo")) )
g.add( (donna, FOAF.name, Literal("Donna Fales")) )
g.add( (donna, FOAF.mbox, URIRef("mailto:donna@example.org")) )

print(g.serialize(format='turtle').decode())

## Assignment part 1: your own webapplication. 

1. in the file 'yourRDF.ttl', replace the four example statements about animals with <b>at least 20 assertions</b> about the domain of your choice. The namespace is already set, so you can use that one or change it if you wish. Note that you should adhere to the turtle format (the easiest format to read)!
2. read in your statements below, and query your triples using basic triple matching, as you've learned above. 